## 1. 環境セットアップ

In [ ]:
# GPU確認
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# 必要なパッケージのインストール
!pip install -q scikit-learn pandas numpy matplotlib seaborn tqdm joblib pillow

## 2. rclone セットアップ（データ同期用）

rcloneを使用してGoogle Driveと同期します。

In [ ]:
# rcloneのインストール
!curl https://rclone.org/install.sh | sudo bash
!rclone version

In [ ]:
# rclone設定（初回のみ実行）
# Google Driveの認証を行います

from google.colab import auth
auth.authenticate_user()

# rclone設定ファイルを作成
import os
os.makedirs('/root/.config/rclone', exist_ok=True)

# Google Drive リモート設定
rclone_config = """
[gdrive]
type = drive
scope = drive
token = {"access_token":"","token_type":"Bearer","refresh_token":"","expiry":""}
team_drive = 
"""

with open('/root/.config/rclone/rclone.conf', 'w') as f:
    f.write(rclone_config)

print("rclone configured for Google Drive")
print("\n設定を確認:")
!rclone config show

In [ ]:
# 手動設定が必要な場合はこちら（上のセルでうまくいかない場合）
# !rclone config
# 対話形式で設定してください：
# 1. n (new remote)
# 2. name: gdrive
# 3. Storage: 15 (Google Drive)
# 4-13: すべてデフォルト（Enter連打）
# 14: Use web browser to automatically authenticate: y
# ブラウザが開くので認証

## 3. Git リポジトリのクローン（ソースコード取得）

GitHubからソースコードをクローンします。

In [ ]:
# プロジェクトディレクトリ設定
import os
from pathlib import Path

PROJECT_ROOT = Path('/content/atma_22_ca')

# GitHubリポジトリURL（自分のリポジトリに変更してください）
GITHUB_REPO = "https://github.com/YOUR_USERNAME/atma_22_ca.git"  # ★要変更★

# すでにクローン済みの場合は削除
if PROJECT_ROOT.exists():
    !rm -rf {PROJECT_ROOT}

# クローン実行
!git clone {GITHUB_REPO} {PROJECT_ROOT}

# 作業ディレクトリを変更
os.chdir(PROJECT_ROOT)

print(f"\nCloned to: {PROJECT_ROOT}")
print(f"Current directory: {os.getcwd()}")

# ファイル確認
!ls -la

In [ ]:
# プライベートリポジトリの場合は Personal Access Token を使用
# GitHub → Settings → Developer settings → Personal access tokens → Generate new token
# repo権限を付与してトークンをコピー

# GITHUB_TOKEN = "ghp_xxxxxxxxxxxxx"  # ★トークンを貼り付け★
# GITHUB_REPO = f"https://{GITHUB_TOKEN}@github.com/YOUR_USERNAME/atma_22_ca.git"

# 上のセルを再実行

## 4. データ同期（rclone）

Google Driveからデータをダウンロードします。

In [ ]:
# Google Drive上のデータパス（自分の環境に合わせて変更）
GDRIVE_DATA_PATH = "gdrive:atma22/data"  # ★要変更★

# ローカルのデータディレクトリ
LOCAL_DATA_PATH = PROJECT_ROOT / "data"
LOCAL_DATA_PATH.mkdir(exist_ok=True)

print(f"Syncing data from Google Drive...")
print(f"  Remote: {GDRIVE_DATA_PATH}")
print(f"  Local:  {LOCAL_DATA_PATH}")
print()

# データをダウンロード（初回は時間がかかります）
!rclone sync -P {GDRIVE_DATA_PATH} {LOCAL_DATA_PATH}

print("\nData sync completed!")
print("\nDirectory structure:")
!tree {LOCAL_DATA_PATH} -L 3 || find {LOCAL_DATA_PATH} -maxdepth 3 -type d

In [ ]:
# データファイルの確認
train_meta = LOCAL_DATA_PATH / 'input' / 'atmaCup22_metadata' / 'train_meta.csv'
test_meta = LOCAL_DATA_PATH / 'input' / 'atmaCup22_metadata' / 'test_meta.csv'
image_dir = LOCAL_DATA_PATH / 'input' / 'images'

print("Data files check:")
print(f"  train_meta.csv: {'✓' if train_meta.exists() else '✗ MISSING'}")
print(f"  test_meta.csv: {'✓' if test_meta.exists() else '✗ MISSING'}")
print(f"  images/: {'✓' if image_dir.exists() else '✗ MISSING'}")

if image_dir.exists():
    n_images = len(list(image_dir.glob('*.jpg')))
    print(f"  Number of images: {n_images:,}")
    
# 必要なディレクトリを作成
(LOCAL_DATA_PATH / 'features').mkdir(exist_ok=True)
(LOCAL_DATA_PATH / 'submission').mkdir(exist_ok=True)
(PROJECT_ROOT / 'models').mkdir(exist_ok=True)
(PROJECT_ROOT / 'logs').mkdir(exist_ok=True)

## 5. モジュールのインポート

In [ ]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime

# パスを追加
sys.path.append(str(PROJECT_ROOT))

# プロジェクトモジュール
from configs.config import *
from src.model_resnet import ModelResNet50
from src.runner import Runner
from src.util import Logger, Metric, Validation

print("Imports successful!")

## 6. データ読み込み

In [ ]:
# メタデータ読み込み
df_train = pd.read_csv(DIR_INPUT / 'atmaCup22_metadata' / 'train_meta.csv')
df_test = pd.read_csv(DIR_INPUT / 'atmaCup22_metadata' / 'test_meta.csv')

# グループ列作成（CV用）
df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)

print(f"Train: {len(df_train):,} samples")
print(f"Test:  {len(df_test):,} samples")
print(f"Players: {df_train['label_id'].nunique()}")
print(f"Groups: {df_train['group'].nunique()}")

# サンプル表示
display(df_train.head())

## 7. CV戦略の確認（オプション）

In [ ]:
# リークチェック（時間がかかる場合はスキップ可）
validator = Validation.create_validator(method='group', n_splits=5)

result = Validation.check_group_leak(
    validator=validator,
    X=df_train,
    y=df_train['label_id'].values,
    groups=df_train['group'].values,
    verbose=True
)

print(f"\nLeak detected: {result['has_leak']}")

## 8. モデル学習（5-Fold CV）

ResNet50 + Prototype法でベースライン学習

In [ ]:
# 実行名（タイムスタンプ付き）
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
run_name = f'resnet50_baseline_{timestamp}'

# パラメータ設定
params = {
    'model_name': 'resnet50',
    'method': 'prototype',  # 'prototype' or 'knn'
    'k': 5,
    'threshold': 0.5,
    'min2_threshold': 0.3,
    'batch_size': 64,  # Colab GPUなら64-128推奨
    'num_workers': 2,  # Colabでは2-4推奨
    'use_cache': True,
}

# CV設定
cv_setting = {
    'method': 'group',
    'group_col': 'group',
    'n_splits': 5,
    'shuffle': True,
    'random_state': 42,
}

# Logger作成
logger = Logger(str(DIR_LOG / run_name))

print(f"Run name: {run_name}")
print(f"Parameters: {params}")
print(f"CV: {cv_setting['n_splits']}-fold {cv_setting['method']}")

In [ ]:
# Runner作成
runner = Runner(
    run_name=run_name,
    model_cls=ModelResNet50,
    params=params,
    df_train=df_train,
    df_test=df_test,
    cv_setting=cv_setting,
    logger=logger
)

print("Runner created successfully!")

In [ ]:
# CV学習実行（時間がかかります: 初回は特徴抽出で1-2時間程度）
# FeatureCacheが有効なので、2回目以降は高速化されます
scores = runner.train_cv()

print(f"\n{'='*80}")
print(f"Final CV Scores:")
for i, score in enumerate(scores):
    print(f"  Fold {i}: {score:.6f}")
print(f"  Mean: {np.mean(scores):.6f} ± {np.std(scores):.6f}")
print(f"{'='*80}")

## 9. テストデータの予測

In [ ]:
# CV学習済みモデルでアンサンブル予測
submission = runner.predict_cv()

# 予測分布の確認
print("\nPrediction distribution:")
print(submission['label_id'].value_counts().sort_index())

# unknown(-1)の割合
unknown_ratio = (submission['label_id'] == -1).sum() / len(submission)
print(f"\nUnknown predictions: {unknown_ratio*100:.2f}%")

## 10. 提出ファイルの保存

In [ ]:
# 提出ファイル保存
submission_path = runner.save_submission(submission, suffix='cv_ensemble')

print(f"\nSubmission file saved: {submission_path}")

## 11. 結果のバックアップ（rclone）

学習結果をGoogle Driveにアップロードします。

In [ ]:
# モデル・ログ・提出ファイルをバックアップ
GDRIVE_BACKUP_PATH = "gdrive:atma22"  # ★要変更★

print("Backing up results to Google Drive...")
print()

# モデルをアップロード
print("[1/4] Uploading models...")
!rclone sync -P {PROJECT_ROOT}/models {GDRIVE_BACKUP_PATH}/models

# ログをアップロード
print("\n[2/4] Uploading logs...")
!rclone sync -P {PROJECT_ROOT}/logs {GDRIVE_BACKUP_PATH}/logs

# 提出ファイルをアップロード
print("\n[3/4] Uploading submissions...")
!rclone sync -P {PROJECT_ROOT}/data/submission {GDRIVE_BACKUP_PATH}/data/submission

# 特徴量キャッシュをアップロード（大きいので注意）
print("\n[4/4] Uploading feature cache...")
!rclone sync -P {PROJECT_ROOT}/data/features {GDRIVE_BACKUP_PATH}/data/features

print("\n✓ Backup completed!")
print(f"Results are saved to: {GDRIVE_BACKUP_PATH}")

In [ ]:
# 提出ファイルを直接ダウンロードする場合
from google.colab import files

# 最新の提出ファイルを取得
import glob
submission_files = sorted(glob.glob(str(DIR_SUBMISSIONS / 'submission_*.csv')))
if submission_files:
    latest_submission = submission_files[-1]
    print(f"Downloading: {latest_submission}")
    files.download(latest_submission)
else:
    print("No submission files found.")

## 12. Git ワークフロー（オプション）

コードに変更を加えた場合のGit操作

In [ ]:
# 変更確認
!git status

In [ ]:
# 最新のコードを取得（ローカルで変更があった場合）
!git pull origin main  # ブランチ名を適宜変更

In [ ]:
# 変更をコミット＆プッシュ（必要な場合）
# !git add .
# !git commit -m "Colab training results"
# !git push origin main

## 13. 運用ガイド・Tips

### 初回セットアップ
1. ローカルでGitリポジトリを作成・push
2. ローカルでrcloneを設定し、データをGoogle Driveにアップロード
3. このノートブックで全セルを実行

### 2回目以降の実行
1. セル3（Git clone）から実行
2. セル4（rclone sync）でデータ同期（差分のみ）
3. 学習実行
4. セル11（rclone backup）で結果をアップロード

### ローカル環境との同期
```bash
# ローカルからGoogle Driveへアップロード
rclone sync ./data/input gdrive:atma22/data/input

# Google Driveからローカルへダウンロード
rclone sync gdrive:atma22/models ./models
rclone sync gdrive:atma22/logs ./logs
rclone sync gdrive:atma22/data/submission ./data/submission
```

### メモリ管理
- Colab無料版のGPUメモリは限られています
- `batch_size`を調整してください（64-128推奨）
- メモリエラーが出たら、ランタイム → ランタイムを再起動

### 実行時間
- **初回実行**: 特徴抽出に1-2時間（全画像を処理）
- **2回目以降**: キャッシュ利用で数分（学習のみ）
- 特徴量キャッシュは `data/features/` に保存され、rcloneでバックアップされます

### Git管理のベストプラクティス
- **ソースコード**: src/, configs/ → Git管理
- **データ**: data/input/ → rclone（.gitignore）
- **モデル**: models/ → rclone（.gitignore）
- **ログ**: logs/ → rclone（.gitignore）
- **特徴量キャッシュ**: data/features/ → rclone（.gitignore）
- **ノートブック**: notebooks/ → Git管理（出力はクリア）

### .gitignore 推奨設定
```
data/
models/
logs/
__pycache__/
*.pyc
.ipynb_checkpoints/
```

### 改善案
1. `params['model_name']` を `'efficientnet_b0'` に変更
2. `params['method']` を `'knn'` に変更してKNN法を試す
3. Data Augmentation の追加（dataset_image.py）
4. ArcFace導入で識別性能向上

### トラブルシューティング
- **GPU not found**: ランタイム → ランタイムのタイプを変更 → GPU
- **rclone auth error**: セル2のrclone設定を再実行、または手動設定
- **Git clone error**: GITHUB_REPOのURLを確認、プライベートならトークン使用
- **File not found**: rclone syncが完了しているか確認
- **Out of memory**: batch_sizeを32や16に減らす
- **Import error**: Gitリポジトリにすべてのソースファイルがpushされているか確認